## <h1 style="color: green;">LAB 02: WORKING WITH TIME-SERIES DATA</h1>

#### Import thư viện và dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
df = pd.read_csv('Data/Road Accident Data.csv')

### 1. Thu Thập Dữ Liệu (Data Collection)

### 2. Khám Phá Dữ Liệu (Data Exploration)

### Data preprocessing

In [ ]:
df.info()

In [ ]:
df['Vehicle_Type'].unique()

In [ ]:
category_mapping = {
    'Car': 'Car',
    'Taxi/Private hire car': 'Taxi',
    'Motorcycle over 500cc': 'Motorcycle',
    'Van / Goods 3.5 tonnes mgw or under': 'Van',
    'Goods over 3.5t. and under 7.5t': 'Other',
    'Motorcycle 125cc and under': 'Motorcycle',
    'Motorcycle 50cc and under': 'Motorcycle',
    'Bus or coach (17 or more pass seats)': 'Bus',
    'Goods 7.5 tonnes mgw and over': 'Other',
    'Other vehicle': 'Other',
    'Motorcycle over 125cc and up to 500cc': 'Motorcycle',
    'Agricultural vehicle': 'Other',
    'Minibus (8 - 16 passenger seats)': 'Other',
    'Pedal cycle': 'Pedal cycle',
    'Ridden horse': 'Ridden horse'
}

# Map values in 'Vehicle_Type' column to the specified categories
df['Vehicle_Type'] = df['Vehicle_Type'].map(category_mapping)

In [ ]:
# Tạo cột "Year" và "Month" để phân tích xu hướng theo năm và tháng
df['Year'] = df['Accident Date'].dt.year
df['Month'] = df['Accident Date'].dt.month
df['Day'] = df['Accident Date'].dt.day

In [ ]:
print(type(accidents_by_month))

### Phân tích biểu đồ các cột đôi

In [ ]:
# Vẽ biểu đồ đường theo tháng
plt.figure(figsize=(12, 6))
accidents_by_month['Date'] = pd.to_datetime(accidents_by_month['Year'].astype(str) + '-' + accidents_by_month['Month'].astype(str), format='%Y-%m')
sns.lineplot(x='Date', y='Accidents', data=accidents_by_month)
plt.title('Tổng số tai nạn giao thông theo tháng')
plt.xlabel('Thời gian')
plt.ylabel('Số tai nạn')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Vẽ biểu đồ đường theo ngày
plt.figure(figsize=(12, 6))
sns.lineplot(x='Accident Date', y='Accidents', data=accidents_by_day)
plt.title('Tổng số tai nạn giao thông theo ngày')
plt.xlabel('Ngày')
plt.ylabel('Số tai nạn')
# plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Chuyển dữ liệu thành một chuỗi thời gian theo tháng
accidents_by_month['Date'] = pd.to_datetime(accidents_by_month['Year'].astype(str) + '-' + accidents_by_month['Month'].astype(str), format='%Y-%m')
accidents_by_month.set_index('Date', inplace=True)

# Decompose chuỗi thời gian
decomposition = seasonal_decompose(accidents_by_month['Accidents'], model='additive', period=12)

# Vẽ decomposition
plt.figure(figsize=(12, 8))
decomposition.plot()
plt.tight_layout()
plt.show()

Có sự khác biệt nào về xu hướng và chu kỳ của số lượng tai nạn trong các khu vực đô thị và nông thôn không?

In [ ]:
# Tách dữ liệu thành hai nhóm: Đô thị và Nông thôn
urban_accidents = df[df['Urban_or_Rural_Area'] == 'Urban']
rural_accidents = df[df['Urban_or_Rural_Area'] == 'Rural']

# Nhóm số tai nạn theo thời gian (tháng) cho từng nhóm
urban_accidents_by_month = (
    urban_accidents.groupby(['Year', 'Month'])
    .size()
    .reset_index(name='Accidents')
)
rural_accidents_by_month = (
    rural_accidents.groupby(['Year', 'Month'])
    .size()
    .reset_index(name='Accidents')
)

# Thêm cột "Date" cho chuỗi thời gian
urban_accidents_by_month['Date'] = pd.to_datetime(
    urban_accidents_by_month['Year'].astype(str) + '-' + urban_accidents_by_month['Month'].astype(str),
    format='%Y-%m'
)
rural_accidents_by_month['Date'] = pd.to_datetime(
    rural_accidents_by_month['Year'].astype(str) + '-' + rural_accidents_by_month['Month'].astype(str),
    format='%Y-%m'
)

# Đặt cột Date làm chỉ số để phục vụ phân tích chuỗi thời gian
urban_accidents_by_month.set_index('Date', inplace=True)
rural_accidents_by_month.set_index('Date', inplace=True)


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Phân tích decomposition cho khu vực đô thị
urban_decompose = seasonal_decompose(urban_accidents_by_month['Accidents'], model='additive', period=12)
rural_decompose = seasonal_decompose(rural_accidents_by_month['Accidents'], model='additive', period=12)

# Vẽ decomposition plot cho khu vực đô thị
plt.figure(figsize=(12, 8))
urban_decompose.plot()
plt.suptitle('Decomposition Plot - Khu vực Đô thị', fontsize=16)
plt.tight_layout()
plt.show()

# Vẽ decomposition plot cho khu vực nông thôn
plt.figure(figsize=(12, 8))
rural_decompose.plot()
plt.suptitle('Decomposition Plot - Khu vực Nông thôn', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# Tổng hợp dữ liệu để vẽ Stacked Area Chart
combined_accidents = pd.DataFrame({
    'Date': urban_accidents_by_month.index,
    'Urban': urban_accidents_by_month['Accidents'],
    'Rural': rural_accidents_by_month['Accidents']
}).set_index('Date')

# Vẽ biểu đồ Stacked Area Chart
plt.figure(figsize=(12, 6))
combined_accidents.plot.area(stacked=True, alpha=0.6, figsize=(12, 6))
plt.title('Phân bổ số lượng tai nạn giữa Đô thị và Nông thôn theo thời gian', fontsize=16)
plt.xlabel('Thời gian')
plt.ylabel('Số lượng tai nạn')
plt.legend(['Đô thị', 'Nông thôn'])
plt.tight_layout()
plt.show()


### 3. Khám Phá Các Mối Quan Hệ Trong Dữ Liệu

#### 3.1 Khám phá các mối quan hệ đơn thuộc tính

#### 3.2 Khám phá các mối quan hệ đa thuộc tính

### 3.3 Phân tích sử dụng mô hình Machine learning